In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from evidently import ColumnMapping
from evidently.test_suite import TestSuite
from evidently.test_preset import DataStabilityTestPreset

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, DataQualityPreset, ClassificationPreset

print("Hello world")

/Users/vishwasgowda/Desktop/Projects/ML/predictive-maintenance/venv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/vishwasgowda/Desktop/Projects/ML/predictive-maintenance/venv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Use

Hello world


/Users/vishwasgowda/Desktop/Projects/ML/predictive-maintenance/venv/lib/python3.11/site-packages/umap/umap_.py:660: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()


In [2]:
ref_data = pd.read_csv("../data/processed/preprocessed.csv")
cur_data = pd.read_csv("../data/processed/test.csv")

In [3]:
ref_data['Machine failure'].value_counts()

Machine failure
1    38617
0    19295
Name: count, dtype: int64

In [4]:
ref_X = ref_data.drop(["Machine failure", "type_of_failure"], axis=1)
ref_y = ref_data["Machine failure"]

cur_X = cur_data.drop(["Machine failure", "type_of_failure"], axis=1)
cur_y = cur_data["Machine failure"]

In [5]:
ref_X_train, ref_X_test, ref_y_train, ref_y_test = train_test_split(ref_X, ref_y, test_size=0.2, random_state=42)

In [6]:
ref_y_test

16971    1
53134    1
46497    0
9584     0
1566     0
        ..
32887    1
35872    1
4623     0
6231     0
40380    0
Name: Machine failure, Length: 11583, dtype: int64

In [7]:
cur_X

,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Air temperature [c],Process temperature [c]
0,0.0,0.097158,0.671322,0.905666,0.408986,0.564773
1,1.0,0.190744,0.506415,0.868395,0.530999,0.726567
2,0.0,0.255629,0.439163,0.855824,0.221831,0.302180
3,2.0,0.175786,0.542582,0.241107,0.315217,0.382716
4,0.0,0.242066,0.447246,0.853494,0.215074,0.301561
...,...,...,...,...,...,...
11578,0.0,0.731702,0.116372,0.468371,0.362275,0.509396
11579,2.0,0.162677,0.552511,0.953051,0.635000,0.527050
11580,2.0,0.178683,0.659309,0.853205,0.446224,0.395711
11581,0.0,0.296275,0.310440,0.577075,0.206522,0.333333


In [8]:
ref_y.value_counts()

Machine failure
1    38617
0    19295
Name: count, dtype: int64

In [9]:
rf = RandomForestClassifier()
rf.fit(ref_X_train.values, ref_y_train.values)



RandomForestClassifier()

## Using predict

In [10]:
ref_pred = rf.predict(ref_X_test)
ref_pred = pd.DataFrame(ref_pred, columns=["Prediction"])
cur_pred = rf.predict(cur_X)
cur_pred = pd.DataFrame(cur_pred, columns=["Prediction"])


/Users/vishwasgowda/Desktop/Projects/ML/predictive-maintenance/venv/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/Users/vishwasgowda/Desktop/Projects/ML/predictive-maintenance/venv/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [11]:
ref_X_test.reset_index(inplace=True, drop=True)
ref_y_test.reset_index(inplace=True, drop=True)
ref_merged = pd.concat([ref_X_test, ref_y_test], axis=1)
ref_merged = pd.concat([ref_merged, ref_pred], axis=1)
ref_merged

,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Air temperature [c],Process temperature [c],Machine failure,Prediction
0,0.0,0.101494,0.657349,0.291770,0.684783,0.543210,1,1
1,0.0,0.229842,0.485903,0.845072,0.716679,0.642380,1,1
2,0.0,0.148023,0.656074,0.412683,0.545314,0.586753,0,0
3,0.0,0.210128,0.515110,0.067194,0.434783,0.592593,0,0
4,1.0,0.136787,0.563187,0.584980,0.293478,0.296296,0,0
...,...,...,...,...,...,...,...,...
11578,0.0,0.164635,0.763735,0.604492,0.735854,0.678621,1,1
11579,1.0,0.085889,0.890437,0.105392,0.106468,0.180631,1,1
11580,0.0,0.207800,0.506868,0.707510,0.836957,0.666667,0,0
11581,1.0,0.190338,0.508242,0.584980,0.652174,0.654321,0,0


In [12]:
cur_X.reset_index(inplace=True, drop=True)
cur_y.reset_index(inplace=True, drop=True)
cur_merged = pd.concat([cur_X, cur_y], axis=1)
cur_merged = pd.concat([cur_merged, cur_pred], axis=1)
cur_merged

,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Air temperature [c],Process temperature [c],Machine failure,Prediction
0,0.0,0.097158,0.671322,0.905666,0.408986,0.564773,1,1
1,1.0,0.190744,0.506415,0.868395,0.530999,0.726567,1,1
2,0.0,0.255629,0.439163,0.855824,0.221831,0.302180,1,1
3,2.0,0.175786,0.542582,0.241107,0.315217,0.382716,0,0
4,0.0,0.242066,0.447246,0.853494,0.215074,0.301561,1,1
...,...,...,...,...,...,...,...,...
11578,0.0,0.731702,0.116372,0.468371,0.362275,0.509396,1,1
11579,2.0,0.162677,0.552511,0.953051,0.635000,0.527050,1,1
11580,2.0,0.178683,0.659309,0.853205,0.446224,0.395711,1,1
11581,0.0,0.296275,0.310440,0.577075,0.206522,0.333333,0,0


In [13]:
cm = ColumnMapping()
cm.target = "Machine failure"
cm.prediction = "Prediction"
cm.target_names = ["No Failure", "Machine Failure"]

In [14]:
classification_performance_report = Report(metrics=[
    ClassificationPreset(),
])
classification_performance_report.run(reference_data=ref_merged, current_data=cur_merged, column_mapping = cm)
classification_performance_report.save_html("../reports/classification_performance_report.html")

In [15]:
edfe2

NameError: name 'edfe2' is not defined

## Using Probas

In [ ]:
ref_probas = pd.DataFrame(rf.predict_proba(ref_X_test))
ref_probas.columns = ['No Failure', 'Machine Failure']
cur_probas = pd.DataFrame(rf.predict_proba(cur_X))
cur_probas.columns = ['No Failure', 'Machine Failure']

/Users/vishwasgowda/Desktop/Projects/ML/predictive-maintenance/venv/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning:

X has feature names, but RandomForestClassifier was fitted without feature names

/Users/vishwasgowda/Desktop/Projects/ML/predictive-maintenance/venv/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning:

X has feature names, but RandomForestClassifier was fitted without feature names



In [ ]:
ref_probas

,No Failure,Machine Failure
0,0.00,1.00
1,0.01,0.99
2,1.00,0.00
3,1.00,0.00
4,0.98,0.02
...,...,...
11578,0.00,1.00
11579,0.00,1.00
11580,0.99,0.01
11581,1.00,0.00


In [ ]:
ref_probas.value_counts()

No Failure  Machine Failure
0.00        1.00               5843
1.00        0.00               3177
0.01        0.99                956
0.02        0.98                359
0.99        0.01                263
                               ... 
0.39        0.61                  1
0.74        0.26                  1
0.64        0.36                  1
0.58        0.42                  1
0.51        0.49                  1
Name: count, Length: 94, dtype: int64

In [ ]:
ref_X_test.reset_index(inplace=True, drop=True)
ref_X_test['Attrition'] = ['No Failure' if x == 0 else 'Machine Failure' for x in ref_y_test]
ref_merged = pd.concat([ref_X_test, ref_probas], axis = 1)

cur_X.reset_index(inplace=True, drop=True)
cur_X['Attrition'] = ['No Failure' if x == 0 else 'Machine Failure' for x in cur_y]
cur_merged = pd.concat([cur_X, cur_probas], axis = 1)

In [ ]:
ref_merged

,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Air temperature [c],Process temperature [c],Attrition,No Failure,Machine Failure
0,0.0,0.096677,0.665923,0.609090,0.791311,0.639513,Machine Failure,0.00,1.00
1,0.0,0.250431,0.398949,0.843750,0.590045,0.579791,Machine Failure,0.01,0.99
2,0.0,0.225347,0.467456,0.706391,0.442553,0.477599,No Failure,1.00,0.00
3,0.0,0.210128,0.515110,0.067194,0.434783,0.592593,No Failure,1.00,0.00
4,1.0,0.136787,0.563187,0.584980,0.293478,0.296296,No Failure,0.98,0.02
...,...,...,...,...,...,...,...,...,...
11578,0.0,0.910478,0.071724,0.556012,0.414578,0.559202,Machine Failure,0.00,1.00
11579,1.0,0.106904,0.825379,0.209224,0.604832,0.495969,Machine Failure,0.00,1.00
11580,0.0,0.207800,0.506868,0.707510,0.836957,0.666667,No Failure,0.99,0.01
11581,1.0,0.190338,0.508242,0.584980,0.652174,0.654321,No Failure,1.00,0.00


In [ ]:
cm  = ColumnMapping()
cm.target = 'Attrition'
cm.prediction = ['No Failure', 'Machine Failure']
cm.pos_label = 'No Failure'

In [ ]:
classification_performance_report = Report(metrics=[
    ClassificationPreset(),
])

classification_performance_report.run(reference_data=ref_merged, current_data=cur_merged, column_mapping = cm)
classification_performance_report.save_html("../reports/classification_performance_report.html")